![banner](../logo/banner.png)

# 3.3 Entradas digitales (inputs)

Las entradas digitales son solo 2 formas, recibir un *nivel logico 0* o un *nivel logico 1*, las cuales indican un estado en la entrada y con base a ello tomar desiciones para realizar una o varias tareas.

Las entradas en el microcontrolador son `nivel lógico 1` o `alto`, lo que tendremos en pin a la entrada será un nivel de voltaje de `5V`, en caso de recibir un `0` o `bajo`, es equivalente a `0V`.


Nivel lógico|Digital|Voltaje
:-:|:-:|:-:
0|Bajo|0V
1|Alto|5V

## 3.3.1 Recibiendo datos a la entrada

Para poder leer datos de la tarjeta debemos habilitar unas opciones más, de lo contrario no podrá leer saber si ha recibido informacion.

Debemos configuar que será entrada el *pin* y ademas activar un iterador en la libreria, el cual hace posible leer datos de entrada.

```python
# importamos los elementos necesarios
from pyfirmata import Arduino, util, INPUT

PORT = 'COM'
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
PIN = 2
board.digital[PIN].mode = INPUT 
```
---

**Ejemplo 1: Haremos un circuito que imprima el valor si el boton esta siendo presionado**

```python
from pyfirmata import Arduino, util, INPUT
from time import sleep

PORT = 'COM'
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
PIN = 2
board.digital[PIN].mode = INPUT 

while True:
    boton = board.digital[PIN].read() # leemos el valor del pin
    print(boton) # mandamos a imprimir el valor por la terminal
    sleep(0.5) # damos un tiempo entre pulsaciones
```

![input 1](imgs/input_1.png)

## **Otra forma de escribir el programa anterior**

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
board.get_pin('d:2:i')

while True:
    print(board.digital[2].read())
    sleep(0.25)
```

## 3.3.2 Pull-down vs Pull-up

Existen dos formas de conexion de un boton hacia una entrada digital, una nos da un valor de un 1 por default (pull-up), mientras que la otra nos da un valor por default de 0 (pull-down)

![pull](imgs/pulls.png)

## Modo de configuracion de pines `get_pin(t:p:m)`

Existe otra manera de configurar los pines, es con el método `get_pin()`, el valor se pasa como string `str` separados por dos puntos `:`.

**Tipo**

- `a` : analogico
- `d` : digital

**Número de pin en la tarjeta:**

- `pin`: del 2 al 13
    
**Modo:**

- `i`: Entrada (input)
- `o`: Salida (output)
- `p`: PWM (salida analogica)

**Ejemplos:**

```python
# pin 2 (D2) como salida de digital
pin2 = board.get_pin('d:2:o')
pin2.write(1)

# pin 4 (D4) como entrada de digital
pin4 = board.get_pin('d:4:i')
pin4.write(1)

# pin 0 (A0) como entrada analogica
analog_0 = board.get_pin('a:0:i')
analog_0.read()

# pin 3 (D3) como salida en PWM (Salida analogica)
pin3 = board.get_pin('d:3:p')
pin3.write(0.6)
```

---

Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)

[Mecatrónica 85](https://mecatronica85.com/)

[fb/mecatronica85](https://www.facebook.com/mecatronica85)